# Libraries

In [1]:
import os
import warnings
import xarray as xr
import rioxarray as rxr

# Suppress specific warning for cleaner output
warnings.filterwarnings('ignore', category=UserWarning, module='rioxarray._io')

# Directories

In [33]:
# Set the base directory with an environment variable, fallback to a relative path
main_dir = os.getenv('PROJECT_DIR', os.path.join(os.getcwd(), 'data', 'ANIN', 'Generating Indices', 'VCI', 'NDVI-16', 'Trial', 'Tiles', 'Crop_mask_new'))

# Open the Phenology image
Phenology = rxr.open_rasterio(os.path.join(main_dir, 'Phenology.tif'), masked=True)
Land_cover_CCI = rxr.open_rasterio(os.path.join(main_dir, 'LC_300m.tif'), masked=True)
Land_cover_S3 = rxr.open_rasterio(os.path.join(main_dir, 'LC_South Africa300.tif'), masked=True)

# Reproject Land_cover_S3 to match Land_cover_CCI
Land_cover_S3 = Land_cover_S3.rio.reproject_match(Land_cover_CCI)

# Print paths for verification
print('Main Directory:', main_dir)
print('Phenology Path:', os.path.join(main_dir, 'Phenology.tif'))
print('Land Cover CCI Path:', os.path.join(main_dir, 'LC_300m.tif'))
print('Land Cover S3 Path:', os.path.join(main_dir, 'LC_South Africa300.tif'))


Tiff files of phenology contain value V expressed over a 3-year period, from 1 to 108 dekads (ten-day periods). This allows treating the circular variable dekad as an ordinal variable. The timing T is expressed in 1-36 dekad units (calendar year): i) if value V <= 36, then T=V; ii) if 36 < V <= 72, then T= V-36; iii) if V > 72, then T=V-72.

In [9]:
# This code adjusts all values to be within 0 to 36, representing each dekad in a year
Phenology_new = xr.where((Phenology > 36) & (Phenology <= 72), Phenology - 36, Phenology)
Phenology_new = xr.where(Phenology_new > 72, Phenology_new - 72, Phenology_new)

In [11]:
# Accumulate dekads into months
count = 0
P = Phenology_new
for i in range(12):
    P = xr.where((Phenology_new > count) & (Phenology_new <= (3 + count)), 1 + i, P)
    count += 3

In [ ]:
# List of month names for file outputs
months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September',
          'October', 'November', 'December']

for month in range(12):
    Growing_season = (xr.where(
        ((P[0] == month + 1) | (P[1] == month + 1) | ((P[0] > month + 1) & (P[1] < P[0]) & (P[1] > month + 1)) | ((P[0] < month + 1) & (P[1] > month + 1)))
        | ((P[2] == month + 1) | (P[3] == month + 1) | ((P[2] > month + 1) & (P[3] < P[2]) & (P[3] > month + 1)) | ((P[2] < month + 1) & (P[3] > month + 1))), 1, 0)
    )

    Crop_mask = xr.where((Growing_season == 1) & (((Land_cover_S3 >= 11) & (Land_cover_S3 <= 30)) | (Land_cover_CCI == 4)), 1, 0)
    Crop_mask = Crop_mask.where(Crop_mask != 0)
    Crop_mask = Crop_mask.transpose('band', 'y', 'x')
    Crop_mask = Crop_mask.rio.write_crs(Land_cover_CCI.rio.crs)

    output_path = os.path.join(main_dir, 'Crop_mask', f'{months[month]}.tif')
    Crop_mask.rio.to_raster(output_path)